In [1]:
import os
cur_path = os.getcwd()
path = os.path.abspath(os.path.join(cur_path,os.pardir))
os.chdir(path)

In [10]:
from model.m26_prob import MultiDecoderCondVAE
from loss.l26oss_all import integrated_loss_fn

In [5]:
import torch
train_loader = torch.load('torch/pre_retrain_loader.pt',weights_only= False)
val_loader = torch.load('torch/pre_reval_loader.pt',weights_only=False)
test_loader = torch.load('torch/pre_retest_loader.pt',weights_only=False)

In [9]:
x,c = next(iter(train_loader))
x_dim = x.shape[1]
c_dim = c.shape[1]
model = MultiDecoderCondVAE(x_dim,c_dim,z_dim = 8)
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)

In [ ]:
epochs = 300
for epoch in range(1,epochs+1):
    tot_train_loss = 0
    model.train()
    optimizer.zero_grad()
    bce_logit,binary_out,x_hat,z_mu,z_logvar = model(x,c)
    loss_dict = integrated_loss_fn(bce_logit,x_hat, x,z_mu,z_logvar)
    loss_dict['loss'].backward()
    tot_train_loss +=loss_dict['loss'].item()
    optimizer.step()
    
    #validation부분
    model.eval()
    v_loss = 0
    total_v_loss = 0
    with torch.no_grad():
        for v_x,v_c in val_loader:
            v_bce_logit,v_binary_out,v_x_hat,v_mu,v_logvar = model(v_x,v_c)
            loss_dict = integrated_loss_fn(v_bce_logit, v_x_hat, v_x,v_mu,v_logvar)
            v_loss += loss_dict['loss'].item()
            tot_v_loss =v_loss
    avg_train_loss = tot_train_loss/len(train_loader)
    avg_val_loss = tot_v_loss/len(val_loader)

    